In [25]:
import json
import requests as req
from bs4 import BeautifulSoup as bs
from urllib.parse import urlencode

In [26]:
SCRAPEOPS_API_KEY = '923784cf-9bce-4662-b322-db42811584fa'

def scrapeops_url(url):
    payload = {'api_key': SCRAPEOPS_API_KEY, 'url': url, 'country': 'us'}
    proxy_url = 'https://proxy.scrapeops.io/v1/?' + urlencode(payload)
    return proxy_url

In [27]:
def scrape_call(search):
    search = search.replace(" ", "+")
    url = 'https://www.walmart.com/search?q='+ search + '&facet=fulfillment_method_in_store%3AIn-store'
    print(url)
    response = req.get(scrapeops_url(url))
    soup = bs(response.content, 'html.parser')
    return soup

In [28]:
class product:
    def __init__(self, name, price, location):
        self.name = name
        self.price = price
        self.location = location

In [42]:
def locate_item(item):
    soup = scrape_call(item)
    script_tag = soup.find("script", {"id": "__NEXT_DATA__"})
    json_blob = json.loads(script_tag.get_text())

    raw_product_data = json_blob["props"]["pageProps"]["initialData"]['searchResult']['itemStacks'][0]['items'][0]

    name = raw_product_data['name']
    price = raw_product_data['price']
    try:
        location = raw_product_data['productLocation'][0]['displayValue']
    except:
        print('No location found for '+ name)
        location = "no location found"

    current = product(name,price,location)
    return(current)

In [41]:
list_of_locations = []
with open('shoppinglist.txt','rb')as f:
    lines = f.readlines()

for item in lines:
    list_of_locations.append(locate_item(item.decode().strip()))

for object in list_of_locations:
    print("Item: " + object.name)
    print("Price: " + str(object.price))
    print("Location: " + object.location)

https://www.walmart.com/search?q=greatvalue+white+bread&facet=fulfillment_method_in_store%3AIn-store
https://www.walmart.com/search?q=frosted+flakes+cereal&facet=fulfillment_method_in_store%3AIn-store
No location found forKellogg's Frosted Flakes Breakfast Cereal, Kids Snacks, Original, 24 Oz, Box
https://www.walmart.com/search?q=white+rice&facet=fulfillment_method_in_store%3AIn-store
Item: Great Value White Sandwich Bread, 20 oz
Price: 1.32
Location: A14
Item: Kellogg's Frosted Flakes Breakfast Cereal, Kids Snacks, Original, 24 Oz, Box
Price: 4.98
Location: no location found
Item: Mahatma Organic White Rice, Extra Long Grain White Rice, 32 oz
Price: 3.92
Location: A11
